In [144]:
import pandas as pd


In [145]:
df_movies = pd.read_csv("./TMDB_movie_dataset_v11.csv")

In [146]:
df_shows = pd.read_csv("./TMDB_tv_dataset_v3.csv")

Movie dataset preprocessing

In [147]:
df_movies.set_index("id", inplace=True)

In [148]:
df_movies.rename(columns={"production_countries":"origin_country"}, inplace=True)

In [149]:
#dropping features where the data is not available. Because I'm using them in the model they can't be filled with a default value
drop_na = ["title","vote_average","vote_count","popularity","genres","overview","keywords"]
df_movies = df_movies.dropna(subset=drop_na)

In [150]:
def determine_type(row):
    origin = row.get('origin_country', '')  # example: "JP", "US", etc.
    genres = row.get('genres', [])
        
    # Check only for 'JP' as origin country
    if origin == None :
        return 'Classic'
    elif origin == 'Japan' and 'Animation' in genres:
        return 'Anime'
    else:
        return 'Classic'

# Create new column "type"
df_movies['content_category'] = df_movies.apply(determine_type, axis=1)

In [151]:
def normalize_genres(genres):
    if isinstance(genres, str):
        return [g.strip() for g in genres.split(",")]
    return genres
df_movies["genres"] = df_movies["genres"].apply(normalize_genres)

In [152]:
df_movies[df_movies["content_category"] == "Anime"]

,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,budget,...,overview,popularity,poster_path,tagline,genres,production_companies,origin_country,spoken_languages,keywords,content_category
id,,,,,,,,,,,,,,,,,,,,,
129,Spirited Away,8.539,14913,Released,2001-07-20,274925095,125,False,/Ab8mkHmkYADjU7wQiOkia9BzGvS.jpg,19000000,...,"A young girl, Chihiro, becomes trapped in a st...",68.024,/39wmItIWsg5sZMyRUHLkWBcuVCM.jpg,NaN,"[Animation, Family, Fantasy]",Studio Ghibli,Japan,Japanese,"witch, parent child relationship, darkness, ba...",Anime
372058,Your Name.,8.514,10303,Released,2016-08-26,357986087,106,False,/dIWwZW7dJJtqC6CgWzYkNVKIUm8.jpg,0,...,High schoolers Mitsuha and Taki are complete s...,68.999,/q719jXXEzOoYaps6babgKnONONX.jpg,It was almost like seeing something out of a d...,"[Romance, Animation, Drama]","CoMix Wave Films, TOHO, KADOKAWA, East Japan M...",Japan,Japanese,"race against time, afterlife, natural disaster...",Anime
4935,Howl's Moving Castle,8.402,8720,Released,2004-09-09,236049757,119,False,/nv5wwZou159v5OC61i4ElR7OqyY.jpg,24000000,...,"Sophie, a young milliner, is turned into an el...",79.888,/6pZgH10jhpToPcf0uvyTCPFhWpI.jpg,The two lived there.,"[Fantasy, Animation, Adventure]","Studio Ghibli, dentsu Music And Entertainment,...",Japan,Japanese,"witch, flying, rain, castle, scarecrow, body e...",Anime
128,Princess Mononoke,8.338,7228,Released,1997-07-12,169000000,134,False,/gl0jzn4BupSbL2qMVeqrjKkF9Js.jpg,23500000,...,"Ashitaka, a prince of the disappearing Emishi ...",54.696,/cMYCDADoLKLbB83g4WnJegaZimC.jpg,The Fate Of The World Rests On The Courage Of ...,"[Adventure, Fantasy, Animation]","Studio Ghibli, Nibariki, dentsu Music And Ente...",Japan,Japanese,"friendship, human vs nature, wolf, fight, wild...",Anime
8392,My Neighbor Totoro,8.073,7124,Released,1988-04-16,41000000,86,False,/fxYazFVeOCHpHwuqGuiqcCTw162.jpg,3700000,...,Two sisters move to the country with their fat...,46.440,/rtGDOeG9LzoerkDGZF9dnVeLppL.jpg,NaN,"[Fantasy, Animation, Family]","Studio Ghibli, Nibariki, Tokuma Shoten",Japan,Japanese,"mother, sibling relationship, village, leave, ...",Anime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719087,Shishigari,0.000,0,Released,2019-07-21,0,17,False,/25xKNl5D3gEdogp6mBKZBXClGDn.jpg,0,...,A gift from the gods of the mountains. Deep in...,0.600,/kS2ViLxgKaPd9Stz2DQ5JPMcmNe.jpg,NaN,"[Animation, Action, Adventure, Fantasy]",Studio Durian,Japan,Japanese,"anime, short film",Anime
726410,Zero Escape: Virtue's Last Reward Special OVA,0.000,0,Released,2011-12-11,0,13,False,NaN,0,...,With his last memory being that of his kidnapp...,1.676,/uYWqOhm3G4OfpqNFRkDQ3yXzJS3.jpg,NaN,"[Animation, Mystery]",GONZO,Japan,Japanese,based on video game,Anime
727877,Mushiking: The Road to the Greatest Champion,0.000,0,Released,2005-12-17,0,50,False,/pGigeoIXQI7oq7lgB2yPXA9tpQx.jpg,0,...,A passionate Mushiking player named Kent Mirai...,0.600,/is4233Cl1ZF8v4x1S7WIfZycTxH.jpg,True Strength is Courage.,"[Animation, Action, Science Fiction]","TOHO, TMS Entertainment",Japan,Japanese,"giant insect, based on video game, anime",Anime


In [153]:
movie_features = ["title","vote_average","vote_count","popularity","keywords","genres","overview","origin_country","content_category"]
df_movie_features = df_movies[movie_features]

Show dataset preprocessing

In [155]:
df_shows.set_index("id", inplace=True)

In [156]:
df_shows.columns

Index(['name', 'number_of_seasons', 'number_of_episodes', 'original_language',
       'vote_count', 'vote_average', 'overview', 'adult', 'backdrop_path',
       'first_air_date', 'last_air_date', 'homepage', 'in_production',
       'original_name', 'popularity', 'poster_path', 'type', 'status',
       'tagline', 'genres', 'created_by', 'languages', 'networks',
       'origin_country', 'spoken_languages', 'production_companies',
       'production_countries', 'episode_run_time'],
      dtype='object')

In [157]:
df_shows.rename(columns={"name":"title"}, inplace=True)

In [158]:
drop_na = ["title","vote_average","vote_count","popularity","genres","overview"]

In [159]:
df_shows = df_shows.dropna(subset=drop_na)

In [163]:
import re

def clean_genres(genre_str):
    if not isinstance(genre_str, str):
        return []
    
    # Replace " & " with ", " to split combined genres
    genre_str = re.sub(r'\s*&\s*', ', ', genre_str)

    # Split by comma and strip spaces
    genres_list = [genre.strip() for genre in genre_str.split(",") if genre.strip()]
    
    return genres_list

In [167]:
df_shows["genres"] = df_shows["genres"].apply(clean_genres)

C:\Users\kayal\AppData\Local\Temp\ipykernel_40012\3192191138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shows["genres"] = df_shows["genres"].apply(clean_genres)


In [169]:
def determine_type(row):
    origin = row.get('origin_country', '')  # example: "JP", "US", etc.
    genres = row.get('genres', [])
        
    # Check only for 'JP' as origin country
    if origin == None :
        return 'Classic'
    elif origin == 'JP' and 'Animation' in genres:
        return 'Anime'
    else:
        return 'Classic'

# Create new column "type"
df_shows['content_category'] = df_shows.apply(determine_type, axis=1)

C:\Users\kayal\AppData\Local\Temp\ipykernel_40012\3050059164.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shows['content_category'] = df_shows.apply(determine_type, axis=1)


In [170]:
show_features = ["title","vote_average","vote_count","popularity","genres","overview","origin_country","content_category"]
df_show_features = df_shows[show_features]

In [172]:
df_show_features["genres"]

id
1399      [Sci-Fi, Fantasy, Drama, Action, Adventure]
71446                                  [Crime, Drama]
66732               [Drama, Sci-Fi, Fantasy, Mystery]
1402      [Action, Adventure, Drama, Sci-Fi, Fantasy]
63174                        [Crime, Sci-Fi, Fantasy]
                             ...                     
238253                                        [Drama]
240698                            [Animation, Comedy]
238374                                       [Comedy]
240609                                        [Drama]
240696                                        [Crime]
Name: genres, Length: 64266, dtype: object

Merge 2 feature datasets

In [173]:
df_show_features["type"] = "TV"
df_movie_features["type"] = "Movie"
merged_feature_df = pd.concat([df_movie_features, df_show_features],axis=0)

C:\Users\kayal\AppData\Local\Temp\ipykernel_40012\3757340692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_show_features["type"] = "TV"
C:\Users\kayal\AppData\Local\Temp\ipykernel_40012\3757340692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie_features["type"] = "Movie"


In [194]:
merged_feature_df.to_csv("merged_feature_df.csv", index=True)

In [189]:
merged_feature_df[(merged_feature_df["type"] == "Movie") & (merged_feature_df["content_category"] == "Anime")]

,title,vote_average,vote_count,popularity,keywords,genres,overview,origin_country,content_category,type
id,,,,,,,,,,
129,Spirited Away,8.539,14913,68.024,"witch, parent child relationship, darkness, ba...","[Animation, Family, Fantasy]","A young girl, Chihiro, becomes trapped in a st...",Japan,Anime,Movie
372058,Your Name.,8.514,10303,68.999,"race against time, afterlife, natural disaster...","[Romance, Animation, Drama]",High schoolers Mitsuha and Taki are complete s...,Japan,Anime,Movie
4935,Howl's Moving Castle,8.402,8720,79.888,"witch, flying, rain, castle, scarecrow, body e...","[Fantasy, Animation, Adventure]","Sophie, a young milliner, is turned into an el...",Japan,Anime,Movie
128,Princess Mononoke,8.338,7228,54.696,"friendship, human vs nature, wolf, fight, wild...","[Adventure, Fantasy, Animation]","Ashitaka, a prince of the disappearing Emishi ...",Japan,Anime,Movie
8392,My Neighbor Totoro,8.073,7124,46.440,"mother, sibling relationship, village, leave, ...","[Fantasy, Animation, Family]",Two sisters move to the country with their fat...,Japan,Anime,Movie
...,...,...,...,...,...,...,...,...,...,...
719087,Shishigari,0.000,0,0.600,"anime, short film","[Animation, Action, Adventure, Fantasy]",A gift from the gods of the mountains. Deep in...,Japan,Anime,Movie
726410,Zero Escape: Virtue's Last Reward Special OVA,0.000,0,1.676,based on video game,"[Animation, Mystery]",With his last memory being that of his kidnapp...,Japan,Anime,Movie
727877,Mushiking: The Road to the Greatest Champion,0.000,0,0.600,"giant insect, based on video game, anime","[Animation, Action, Science Fiction]",A passionate Mushiking player named Kent Mirai...,Japan,Anime,Movie


In [195]:
new_df = pd.read_csv("merged_feature_df.csv")

C:\Users\kayal\AppData\Local\Temp\ipykernel_40012\2538431142.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv("merged_feature_df.csv")


In [197]:
new_df[(new_df["type"] == "Movie") & (new_df["content_category"] == "Anime")]

,id,title,vote_average,vote_count,popularity,keywords,genres,overview,origin_country,content_category,type
107,129,Spirited Away,8.539,14913,68.024,"witch, parent child relationship, darkness, ba...","['Animation', 'Family', 'Fantasy']","A young girl, Chihiro, becomes trapped in a st...",Japan,Anime,Movie
258,372058,Your Name.,8.514,10303,68.999,"race against time, afterlife, natural disaster...","['Romance', 'Animation', 'Drama']",High schoolers Mitsuha and Taki are complete s...,Japan,Anime,Movie
357,4935,Howl's Moving Castle,8.402,8720,79.888,"witch, flying, rain, castle, scarecrow, body e...","['Fantasy', 'Animation', 'Adventure']","Sophie, a young milliner, is turned into an el...",Japan,Anime,Movie
482,128,Princess Mononoke,8.338,7228,54.696,"friendship, human vs nature, wolf, fight, wild...","['Adventure', 'Fantasy', 'Animation']","Ashitaka, a prince of the disappearing Emishi ...",Japan,Anime,Movie
492,8392,My Neighbor Totoro,8.073,7124,46.440,"mother, sibling relationship, village, leave, ...","['Fantasy', 'Animation', 'Family']",Two sisters move to the country with their fat...,Japan,Anime,Movie
...,...,...,...,...,...,...,...,...,...,...,...
224270,719087,Shishigari,0.000,0,0.600,"anime, short film","['Animation', 'Action', 'Adventure', 'Fantasy']",A gift from the gods of the mountains. Deep in...,Japan,Anime,Movie
224433,726410,Zero Escape: Virtue's Last Reward Special OVA,0.000,0,1.676,based on video game,"['Animation', 'Mystery']",With his last memory being that of his kidnapp...,Japan,Anime,Movie
224529,727877,Mushiking: The Road to the Greatest Champion,0.000,0,0.600,"giant insect, based on video game, anime","['Animation', 'Action', 'Science Fiction']",A passionate Mushiking player named Kent Mirai...,Japan,Anime,Movie
224531,727882,Mushiking: Super Battle Movie ～Altered Beetles...,0.000,0,0.600,"giant insect, based on video game, anime","['Animation', 'Action', 'Fantasy']","Adder, the Fairy of Darkness, schemes to invad...",Japan,Anime,Movie


In [3]:
merged_feature_df[merged_feature_df["title"] == "The Good Place"]

NameError: name 'merged_feature_df' is not defined